# Tarefa 01: Compreendendo o Bagging (Bootstrap Aggregating)

## 1. O Passo a Passo do Bagging

O processo de construção de um modelo de **Ensemble** via Bagging segue um fluxo lógico dividido em três etapas fundamentais:

* **Criação dos Subconjuntos (Bootstrap):** A partir do banco de dados original, são criados múltiplos subconjuntos de dados. Este processo ocorre por meio de sorteios aleatórios de linhas **com reposição**, o que significa que uma mesma observação pode aparecer repetidas vezes em um mesmo subconjunto.
* **Treinamento em Paralelo (Modelagem):** Define-se um algoritmo base (geralmente uma **Árvore de Decisão**) que será treinado de forma independente para cada subconjunto gerado no passo anterior. Como os modelos são processados em paralelo, eles não interagem entre si durante a fase de aprendizagem.
* **Votação ou Média (Agregação):** Para consolidar o resultado final das múltiplas predições:
    * **Classificação:** Utiliza-se a votação majoritária (a classe mais frequente entre os modelos vence).
    * **Regressão:** Calcula-se a média aritmética simples das previsões de todos os modelos.



---

## 2. Explicação Conceitual

O **Bagging** funciona como uma proteção robusta contra erros individuais e ruídos nos dados. Em vez de confiar em um único modelo que pode sofrer de *overfitting* (ajuste excessivo ao ruído), utilizamos a força do conjunto.

Como cada modelo é treinado com uma amostra diferente (**Bootstrap**), erros específicos de uma fatia de dados não comprometem o resultado final, pois são filtrados pelo consenso no momento da **Agregação**. Em termos estatísticos, essa técnica é extremamente eficaz para reduzir a **variância**, gerando uma análise muito mais estável e confiável, especialmente em mercados voláteis como o financeiro.

------------------

# 3. Implementação em Python

## Vamos usar o scikit-learn para criar um modelo de Bagging que tenta classificar se um ativo terá retorno positivo.

In [3]:
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
# 1. Preparando os dados 
# Vamos supor que queremos prever se amanhã o retorno é positivo (1) ou negativo (0)
df_ret = pd.read_csv('dados_ativos_fechamento.csv', index_col=0)
df_ret = df_ret.pct_change().dropna()

# Prever direção da VALE3 com base no retorno de hoje
X = df_ret[['ITUB4.SA', 'PETR4.SA']] # Variáveis explicativas
y = (df_ret['VALE3.SA'] > 0).astype(int) # Alvo: 1 se subiu, 0 se caiu

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. O BAGGING na prática
# Criamos 50 Árvores de Decisão independentes
modelo_bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50, 
    max_samples=0.8, # Cada modelo vê 80% dos dados (Bootstrap)
    random_state=42
)

# 3. Treinamento
modelo_bagging.fit(X_train, y_train)

# 4. Avaliação
previsoes = modelo_bagging.predict(X_test)
print(f"Acurácia do modelo Bagging: {accuracy_score(y_test, previsoes):.2%}")

Acurácia do modelo Bagging: 48.67%


------------------------